# Import Library

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
import warnings
from tqdm.auto import tqdm

# Definisi Kolom Fitur (X) dan Pelabelan (Y)

In [ ]:
# FITUR UNTUK TRAINING MODEL (X) - 15 fitur fundamental
# Mengukur: Kinerja Operasional & Kesehatan Internal
FEATURE_COLUMNS = [
    # Profitability & Efficiency (6 fitur)
    'grossProfits',          # Laba kotor absolut
    'operatingMargins',      # Margin operasional (%)
    'profitMargins',         # Margin laba bersih (%)
    'returnOnEquity',        # ROE (%)
    'returnOnAssets',        # ROA (%)
    'ebitdaMargins',         # Margin EBITDA (%)

    # Growth & Performance (5 fitur)
    'revenueGrowth',         # Pertumbuhan pendapatan (%)
    'earningsGrowth',        # Pertumbuhan laba (%)
    'earningsQuarterlyGrowth', # Pertumbuhan laba kuartalan (%)
    'operatingCashflow',     # Arus kas operasi (absolut)
    'freeCashflow',          # Arus kas bebas (absolut)

    # Financial Health & Stability (4 fitur)
    'debtToEquity',          # Rasio utang/ekuitas
    'currentRatio',          # Rasio lancar
    'quickRatio',            # Rasio cepat
    'totalDebt'              # Total utang absolut
]

# FITUR UNTUK PELABELAN (Y) - 10 fitur valuation & quality
# Mengukur: Persepsi Pasar & Valuasi
LABELING_COLUMNS = [
    # Valuation Multiples (5 fitur)
    'trailingPE',            # P/E Ratio
    'forwardPE',             # Forward P/E
    'priceToBook',           # P/B Ratio
    'priceToSalesTrailing12Months', # P/S Ratio
    'enterpriseToRevenue',   # EV/Revenue

    # Quality & Performance (5 fitur)
    'trailingEps',           # EPS
    'netIncomeToCommon',     # Laba bersih
    'bookValue',             # Nilai buku
    'enterpriseValue',       # Enterprise Value
    'payoutRatio'            # Dividend payout ratio
]

# Gabungkan semua kolom yang dibutuhkan
ALL_NEEDED_COLUMNS = ['symbol'] + FEATURE_COLUMNS + LABELING_COLUMNS

# Fungsi: Mendapatkan Ticker S&P 500

In [ ]:
# Mengambil daftar ticker S&P 500 dari Github

def get_sp500_tickers():
    try:
        url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
        print(f"Mengunduh data S&P 500 dari: {url}")

        df_sp500 = pd.read_csv(url)

        # Ambil ticker dari kolom 'Symbol'
        # Ganti '.' dengan '-' (misal: BRK.B -> BRK-B) agar kompatibel yfinance
        tickers = df_sp500['Symbol'].str.replace('.', '-', regex=False).tolist()

        print(f"Berhasil mengambil {len(tickers)} ticker S&P 500.")
        return tickers

    except Exception as e:
        print(f"Gagal mengambil data ticker S&P 500: {e}")
        return []

# EKSEKUSI
tickers_sp500 = get_sp500_tickers()

Mengunduh data S&P 500 dari: https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv
Berhasil mengambil 503 ticker S&P 500.


# Fungsi: Batch Download Data Saham

In [ ]:
# Mengambil data .info, TAPI HANYA MENYIMPAN fitur yang ada di required_columns.
def get_stock_info_batch_efficient(tickers, required_columns):
    stock_info_list = []
    pbar = tqdm(total=len(tickers), desc="Mengambil Data .info Saham")

    for ticker_str in tickers:
        try:
            stock = yf.Ticker(ticker_str)
            info = stock.info

            # Cek data valid
            if not info or 'symbol' not in info or 'financialCurrency' not in info:
                tqdm.write(f"Data .info tidak lengkap/kosong untuk {ticker_str}, skipping...")
                pbar.update(1)
                continue

            # Buat kamus baru yang hanya berisi data yang kita inginkan
            filtered_info = {}
            for col in required_columns:
                # Gunakan .get() untuk mengambil nilai, default ke np.nan jika tidak ada
                filtered_info[col] = info.get(col, np.nan)

            # Ganti nama 'symbol' ke 'Ticker'
            filtered_info['Ticker'] = filtered_info.pop('symbol')

            stock_info_list.append(filtered_info)

        except Exception as e:
            tqdm.write(f"Gagal mengambil {ticker_str}: {e}")

        pbar.update(1) # Update progress bar

    pbar.close()
    return stock_info_list

# Eksekusi: Scraping Data S&P 500

In [ ]:
# Kolom yang akan kita kirim ke fungsi scraper
# 'symbol' ada di ALL_NEEDED_COLUMNS, 'Ticker' tidak (karena akan di-rename)
COLUMNS_TO_SCRAPE = ALL_NEEDED_COLUMNS

# Jalankan fungsi scraper yang efisien
stock_info_list_sp500 = get_stock_info_batch_efficient(tickers_sp500, COLUMNS_TO_SCRAPE)

print(f"\nBerhasil mengambil data untuk {len(stock_info_list_sp500)} saham S&P 500.")

Mengambil Data .info Saham:   0%|          | 0/503 [00:00<?, ?it/s]

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


Data .info tidak lengkap/kosong untuk WBA, skipping...

Berhasil mengambil data untuk 502 saham S&P 500.


# Inisialisasi DataFrame

In [ ]:
# Konversi list of dictionaries (yang baru dan sudah bersih) ke DataFrame
df_scraped = pd.DataFrame(stock_info_list_sp500)

print(f"Shape data mentah (sudah difilter): {df_scraped.shape}")
print(f"Total kolom: {len(df_scraped.columns)}")
df_scraped.head()

Shape data mentah (sudah difilter): (502, 26)
Total kolom: 26


,grossProfits,operatingMargins,profitMargins,returnOnEquity,returnOnAssets,ebitdaMargins,revenueGrowth,earningsGrowth,earningsQuarterlyGrowth,operatingCashflow,...,forwardPE,priceToBook,priceToSalesTrailing12Months,enterpriseToRevenue,trailingEps,netIncomeToCommon,bookValue,enterpriseValue,payoutRatio,Ticker
0,10172999680,0.24367,0.13700,0.72921,0.07971,0.24818,0.035,-0.375,-0.392,2.541000e+09,...,21.597467,19.584480,3.651062,3.975,6.26,3400999936,8.712,9.867561e+10,0.4609,MMM
1,1475100032,0.18631,0.13851,0.28209,0.13878,0.20477,0.044,0.146,0.099,6.556000e+08,...,16.238327,4.989054,2.402608,2.416,3.71,530500000,13.247,9.251927e+09,0.3666,AOS
2,24665999360,0.19395,0.31880,0.30620,0.06793,0.26793,0.069,0.000,-0.001,9.119000e+09,...,25.050386,4.411152,5.126624,5.253,7.96,13925999616,29.303,2.303186e+11,0.2915,ABT
3,42622001152,0.35497,0.04004,1.37961,0.09585,0.49492,0.091,-0.887,-0.881,2.086000e+10,...,19.150040,-155.377910,6.883270,7.943,1.32,2348000000,-1.495,4.737668e+11,4.9015,ABBV
4,22235400192,0.15220,0.11021,0.25509,0.11183,0.17543,0.073,-0.155,-0.161,1.147440e+10,...,17.595594,4.935114,2.203548,2.171,12.16,7678432768,50.165,1.512715e+11,0.4872,ACN


# Preprocessing: Filtering & Cleaning Data

In [ ]:
# 1. Salin data
df_clean = df_scraped.copy()
print(f"Shape awal (sebelum cleaning): {df_clean.shape}")

# 2. Hapus baris di mana data pelabelan UTAMA tidak ada
key_labeling_cols = ['trailingPE', 'priceToBook', 'trailingEps', 'netIncomeToCommon']
df_clean.dropna(subset=key_labeling_cols, inplace=True)
print(f"Shape setelah dropna pada data pelabelan kunci: {df_clean.shape}")

# 3. Filter data yang tidak logis (penting untuk peringkat)
df_clean = df_clean[df_clean['trailingPE'] > 0] # Hanya P/E positif
df_clean = df_clean[df_clean['priceToBook'] > 0] # Hanya P/B positif
df_clean = df_clean[df_clean['trailingEps'] > 0] # Hanya EPS positif
print(f"Shape setelah filter data tidak logis: {df_clean.shape}")

# 4. Ganti nilai Infinitas (jika ada) dengan NaN
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

print(f"Total baris bersih: {df_clean.shape[0]}")

Shape awal (sebelum cleaning): (502, 26)
Shape setelah dropna pada data pelabelan kunci: (476, 26)
Shape setelah filter data tidak logis: (444, 26)
Total baris bersih: 444


# Pelabelan: Fungsi Peringkat Relatif (Top 25% UQ Score)

In [ ]:
# Menciptakan label 'Undervalued Quality' (Top 25%) berdasarkan skor komposit dari metrik valuasi dan kualitas
def create_labels_relative_ranking(df):
    # 1. Pisahkan metrik berdasarkan logikanya
    # A. Metrik Valuasi (Nilai RENDAH lebih baik)
    valuation_cols = [
        'trailingPE', 'forwardPE', 'priceToBook',
        'priceToSalesTrailing12Months', 'enterpriseToRevenue'
    ]

    # B. Metrik Kualitas (Nilai TINGGI lebih baik)
    quality_cols = ['trailingEps', 'netIncomeToCommon']

    valid_val_cols = [col for col in valuation_cols if col in df.columns]
    valid_qual_cols = [col for col in quality_cols if col in df.columns]

    df_ranks = pd.DataFrame(index=df.index)

    # 2. Hitung Peringkat (Percentile Ranks)

    # Untuk Valuasi -> ascending=False (Nilai Rendah = Peringkat Tinggi 1.0)
    for col in valid_val_cols:
        df_ranks[f'{col}_rank'] = df[col].rank(pct=True, ascending=False)

    # Untuk Kualitas -> ascending=True (Nilai Tinggi = Peringkat Tinggi 1.0)
    for col in valid_qual_cols:
        df_ranks[f'{col}_rank'] = df[col].rank(pct=True, ascending=True)

    # 3. Hitung Skor Komposit
    #    Kita isi NaN di peringkat dengan 0.5 (rata-rata) agar tidak bias
    df['composite_score'] = df_ranks.fillna(0.5).sum(axis=1)

    # 4. Buat Label berdasarkan Peringkat Top 25% (Quantile 0.75)
    quantile_threshold = df['composite_score'].quantile(0.75)
    print(f"Batas skor (Quantile 75%): {quantile_threshold:.4f}")

    # Tentukan kondisi boolean
    is_undervalued_quality = (df['composite_score'] > quantile_threshold)

    # Terapkan label string
    df['Label'] = np.where(is_undervalued_quality,
                           'Undervalued Quality',
                           'Non Undervalued Quality')

    print("\nDistribusi Label:")
    print(df['Label'].value_counts(normalize=True))

    return df

# EKSEKUSI FUNGSI PELABELAN
df_labeled = create_labels_relative_ranking(df_clean.copy())

Batas skor (Quantile 75%): 4.4623

Distribusi Label:
Label
Non Undervalued Quality    0.75
Undervalued Quality        0.25
Name: proportion, dtype: float64


# Menyimpan Dataset ke File CSV

In [ ]:
# Kita simpan Ticker, 15 Fitur (X), dan 1 Label (y)
final_cols_to_save = ['Ticker'] + FEATURE_COLUMNS + ['Label']

# Pastikan semua kolom ada
final_cols_to_save = [col for col in final_cols_to_save if col in df_labeled.columns]

df_final = df_labeled[final_cols_to_save]

# Simpan ke CSV
FILENAME_FINAL = 'sp500_uq_dataset.csv'
df_final.to_csv(FILENAME_FINAL, index=False)

print(f"Dataset final berhasil disimpan di: {FILENAME_FINAL}")

Dataset final berhasil disimpan di: sp500_uq_dataset.csv
